# ai3 for the same thing

## dependencies

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## dataset stuff

### read the dataset

In [6]:
df = pd.read_csv('dataset.csv')
df = df.sample(frac = 1)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
94,63,0,3,135,252,0,2,172,0,0.0,1,0.0,3.0,0
5,56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0
254,43,1,4,115,303,0,0,181,0,1.2,2,0.0,3.0,0
138,35,1,4,120,198,0,0,130,1,1.6,2,0.0,7.0,1
220,41,0,3,112,268,0,2,172,1,0.0,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,54,0,3,108,267,0,2,167,0,0.0,1,0.0,3.0,0
110,61,0,4,145,307,0,2,146,1,1.0,2,0.0,7.0,1
8,63,1,4,130,254,0,2,147,0,1.4,2,1.0,7.0,2
194,68,0,3,120,211,0,2,115,0,1.5,2,0.0,3.0,0


### train_test split

In [9]:
ratio = 0.9 # 90:10 train:test ratio
train_set = df.sample(frac=ratio)

# Dropping all those indexes from the dataframe that exists in the train_set
test_set = df.drop(train_set.index)
train_set.shape, test_set.shape

train_set
test_set

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
53,44,1,2,130,219,0,2,188,0,0.0,1,0.0,3.0,0
146,57,1,4,165,289,1,2,124,0,1.0,2,3.0,7.0,4
266,52,1,4,128,204,1,0,156,1,1.0,2,0.0,?,2
36,43,1,4,120,177,0,2,120,1,2.5,2,0.0,7.0,3
277,39,0,3,138,220,0,0,152,0,0.0,2,0.0,3.0,0
91,62,0,4,160,164,0,2,145,0,6.2,3,3.0,7.0,3
39,61,1,3,150,243,1,0,137,1,1.0,2,0.0,3.0,0
132,29,1,2,130,204,0,2,202,0,0.0,1,0.0,3.0,0
83,68,1,3,180,274,1,2,150,1,1.6,2,0.0,7.0,3
48,65,0,3,140,417,1,2,157,0,0.8,1,1.0,3.0,0


### formating

## ai

### training

### testing

## ui